In [1]:
import pandas as pd
import numpy as np
import os 

path = r'/Users/m31781/Documents/NFL Decision Tree/Second Run'

os.chdir(path)

com_data = pd.read_csv('2020_NFL_Game_Data.csv')
print(com_data.shape)
com_data.head(5)

#Change the stats to season statistics
#Do I need to train include the teams into the data set? 
#train on the entire set
# shape (465, 20)

#get indexs of every team's data 
team_index = com_data['FULL_TEAM_NAME']

#Remove Tm, Week, Opponent Score, Result
rf_model_data = com_data[['FULL_TEAM_NAME', 'Opp', 'TM_Score', 
       'O_1stD', 'O_Tot_Yd', 'O_P_Yd', 'O_R_Yd', 'O_TO', 'D_1stD', 'D_Tot_Yd',
       'D_P_Yd', 'D_R_Yd', 'D_TO', 'Home', 'ADA_Pred_Mean',
       'LOG_Prediction']]

#Change to season statistics
season_stats = ['O_1stD', 'O_Tot_Yd', 'O_P_Yd', 'O_R_Yd', 'O_TO', 'D_1stD', 'D_Tot_Yd',
       'D_P_Yd', 'D_R_Yd', 'D_TO']

rf_model_data[season_stats] = rf_model_data[season_stats] * 16

#get indexs of every team's data 
team_index = com_data['FULL_TEAM_NAME']

#rf_model_data = rf_model_data.drop(['FULL_TEAM_NAME', 'Opp'], axis = 1) #run to remove team names
rf_model_data = pd.get_dummies(rf_model_data)
rf_model_data.head()

(512, 20)


/Users/m31781/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,TM_Score,O_1stD,O_Tot_Yd,O_P_Yd,O_R_Yd,O_TO,D_1stD,D_Tot_Yd,D_P_Yd,D_R_Yd,...,Opp_New Orleans Saints,Opp_New York Giants,Opp_New York Jets,Opp_Philadelphia Eagles,Opp_Pittsburgh Steelers,Opp_San Francisco 49ers,Opp_Seattle Seahawks,Opp_Tampa Bay Buccaneers,Opp_Tennessee Titans,Opp_Washington Football Team
0,17,240,4064,3232,832,32,496,6464,4896,1568,...,0,0,0,0,0,0,0,0,0,0
1,13,272,4432,2768,1664,0,272,5744,2832,2912,...,0,0,0,0,0,1,0,0,0,0
2,7,240,4160,2416,1744,48,336,5648,3744,1904,...,0,0,0,0,0,0,0,0,0,0
3,28,288,5136,3072,2064,0,336,5744,3872,1872,...,0,0,0,0,0,0,0,0,0,0
4,10,336,4560,2592,1968,0,448,7936,5904,2032,...,0,0,0,0,0,0,0,0,0,0


# MODELING

In [2]:
#Random forest
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

#Data preprocessing
X = rf_model_data.drop('TM_Score', axis=1).values
y = rf_model_data['TM_Score'].values.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

RF_model = RandomForestRegressor(n_estimators = 2000)

RF_model.fit(X_train, y_train)

y_pred = RF_model.predict(X_test)

print(mean_absolute_error(y_test, y_pred))

<ipython-input-2-dbea918283e3>:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  RF_model.fit(X_train, y_train)


6.249230769230769


In [23]:
pd.options.display.max_rows = None

importance = RF_model.feature_importances_

feature_names = rf_model_data.drop('TM_Score', axis=1).columns

d = {'Var_Name': feature_names, 'Imp': importance}
df = pd.DataFrame(data=d)
df = df.sort_values(by = ['Imp'], ascending = False).reset_index(drop = True)
df.head(15)

#write a function that returns the scores of two teams playing each other

pd.options.display.max_columns = None

#Create playoff test dataset from season averages 

def Score_Predictor(home_team, away_team):
    team1 = home_team #Home Team
    team2 = away_team #Away Team

    team1_data = rf_model_data[com_data['FULL_TEAM_NAME'] == team1].drop('TM_Score', axis=1).reset_index(drop = True)
    team2_data = rf_model_data[com_data['FULL_TEAM_NAME'] == team2].drop('TM_Score', axis=1).reset_index(drop = True)

    week_slice = slice(11,16)
    
    #1 #Remove if running without team names
    team1_test = pd.DataFrame(team1_data[week_slice].mean(axis = 0)).T #select weeks to use as average
    team1_test
    opp_columns = team1_test.filter(like='Opp').columns

    team1_test[opp_columns] = 0
    team1_test['Opp_' + team2] = 1
    team1_test['Home'] = 0

    #2
    team2_test = pd.DataFrame(team2_data[week_slice].mean(axis = 0)).T #select weeks to use as average
    opp_columns = team2_test.filter(like='Opp').columns

    team2_test[opp_columns] = 0
    team2_test['Opp_' + team1] = 1
    team2_test['Home'] = 0 #change to remove home field advantage

    #head to head matchup
    team1_test[['D_1stD', 'D_Tot_Yd', 'D_P_Yd', 'D_R_Yd', 'D_TO']] = team2_test[['O_1stD', 'O_Tot_Yd', 'O_P_Yd', 'O_R_Yd', 'O_TO']]
    team2_test[['D_1stD', 'D_Tot_Yd', 'D_P_Yd', 'D_R_Yd', 'D_TO']] = team1_test[['O_1stD', 'O_Tot_Yd', 'O_P_Yd', 'O_R_Yd', 'O_TO']]

    X_Playoff_test = pd.concat([team1_test, team2_test])
    #X_Playoff_test = X_Playoff_test.iloc[:, 0:13] #run without team names
    #print(X_Playoff_test)

    scores = RF_model.predict(X_Playoff_test)
    print(team1, 'will score', round(scores[0], 1))
    print(team2, 'will score', round(scores[1], 1))
    
    if scores[0] > scores[1]:
        winner = team1
    else:
        winner = team2
        
    print(winner, 'are the WINNERS!!!!')
    
    return scores, winner

In [4]:
print(y_train.shape)
print(y_test.shape)

print()

(486, 1)
(26, 1)



# ROUND 1

## AFC

In [5]:
#7 at #2
scores, winner = Score_Predictor('Indianapolis Colts', 'Buffalo Bills')

Indianapolis Colts will score 29.4
Buffalo Bills will score 33.1
Buffalo Bills are the WINNERS!!!!


In [6]:
#5 at #4
scores, winner = Score_Predictor('Baltimore Ravens', 'Tennessee Titans')

Baltimore Ravens will score 29.7
Tennessee Titans will score 26.4
Baltimore Ravens are the WINNERS!!!!


In [7]:
#6 at #3
scores, winner = Score_Predictor('Cleveland Browns', 'Pittsburgh Steelers')

Cleveland Browns will score 28.0
Pittsburgh Steelers will score 22.4
Cleveland Browns are the WINNERS!!!!


## NFC

In [8]:
#6 at #3
scores, winner = Score_Predictor('Los Angeles Rams', 'Seattle Seahawks')

Los Angeles Rams will score 22.1
Seattle Seahawks will score 21.1
Los Angeles Rams are the WINNERS!!!!


In [9]:
#5 at #4
scores, winner = Score_Predictor('Tampa Bay Buccaneers', 'Washington Football Team')

Tampa Bay Buccaneers will score 30.2
Washington Football Team will score 21.1
Tampa Bay Buccaneers are the WINNERS!!!!


In [10]:
#7 at #2
scores, winner = Score_Predictor('Chicago Bears', 'New Orleans Saints')

Chicago Bears will score 27.8
New Orleans Saints will score 26.2
Chicago Bears are the WINNERS!!!!


# Round 2

## AFC

In [11]:
#5 at #2
scores, winner = Score_Predictor('Baltimore Ravens', 'Buffalo Bills')

Baltimore Ravens will score 32.2
Buffalo Bills will score 33.5
Buffalo Bills are the WINNERS!!!!


In [12]:
#6 at #1
scores, winner = Score_Predictor('Cleveland Browns', 'Kansas City Chiefs')

Cleveland Browns will score 27.5
Kansas City Chiefs will score 25.0
Cleveland Browns are the WINNERS!!!!


## NFC

In [13]:
#5 at #2
scores, winner = Score_Predictor('Tampa Bay Buccaneers', 'New Orleans Saints')

Tampa Bay Buccaneers will score 29.3
New Orleans Saints will score 28.2
Tampa Bay Buccaneers are the WINNERS!!!!


In [14]:
#6 at #1
scores, winner = Score_Predictor('Los Angeles Rams', 'Green Bay Packers')

Los Angeles Rams will score 21.9
Green Bay Packers will score 27.4
Green Bay Packers are the WINNERS!!!!


# Round 3

## AFC

In [22]:
#AFC Championship Game
scores, winner = Score_Predictor('Buffalo Bills', 'Kansas City Chiefs')

Buffalo Bills will score 33.6
Kansas City Chiefs will score 26.7
Buffalo Bills are the WINNERS!!!!


## NFC

In [17]:
#NFC Championship Game
scores, winner = Score_Predictor('Tampa Bay Buccaneers', 'Green Bay Packers')

Tampa Bay Buccaneers will score 28.2
Green Bay Packers will score 30.1
Green Bay Packers are the WINNERS!!!!


# SUPERBOWL

In [27]:
#Superbowl
scores, winner = Score_Predictor('Tampa Bay Buccaneers', 'Kansas City Chiefs')

Tampa Bay Buccaneers will score 29.4
Kansas City Chiefs will score 25.8
Tampa Bay Buccaneers are the WINNERS!!!!
